In [1]:
import torch
import numpy as np 
import pandas as pd 
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AutoTokenizer, AutoModelForMaskedLM
from scipy.spatial.distance import cosine 
import tokenizers 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy
import os 
import re


2021-12-31 10:24:12.458658: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-31 10:24:12.458724: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [46]:
# vocab_path = os.getcwd()+'/bert_model/vocab_low_data.txt'
# model_path = os.getcwd()+'/blabla/'

In [47]:
# vocab_path

In [48]:
# tokenizers = BertTokenizer.from_pretrained(vocab_path)
# model = BertForMaskedLM.from_pretrained(pretrained_model_name_or_path=model_path, return_dict = True, output_hidden_states = True)

In [2]:
model = AutoModelForMaskedLM.from_pretrained("Shushant/nepaliBERT", output_hidden_states = True, return_dict = True, output_attentions = True)

In [3]:
tokenizers = AutoTokenizer.from_pretrained("Shushant/nepaliBERT")

In [6]:
tokenizers.tokenize("के मौजुदा लोकतान्त्रिक व्यवस्था राज्य पुनःसंरचनासँग जोडिएका हिजोका सवालहरूलाई यथास्थितिमा छोडेर सबल होला ? ठूलो संख्या सडकमा उतार्न नसक्नेको प्रतिरोधी बोली बेवास्तामा परिरहँदा मौजुदा व्यवस्थाबाट न्याय र समता सम्भव छ कसरी भन्न सकिएला ")

['क',
 'मौ',
 '##ज',
 '##दा',
 'लोक',
 '##तान',
 '##तर',
 '##िक',
 'वय',
 '##वस',
 '##था',
 'राज',
 '##य',
 'पन',
 '##ः',
 '##सर',
 '##चना',
 '##सग',
 'जोडिएका',
 'हिजोका',
 'सवाल',
 '##हर',
 '##लाई',
 'यथा',
 '##स',
 '##थिति',
 '##मा',
 'छोड',
 '##र',
 'सबल',
 'होला',
 '?',
 'ठ',
 '##लो',
 'सख',
 '##या',
 'सडकमा',
 'उतार',
 '##न',
 'नसक',
 '##नको',
 'पर',
 '##तिर',
 '##ोध',
 '##ी',
 'बोली',
 'ब',
 '##वास',
 '##तामा',
 'परि',
 '##रहदा',
 'मौ',
 '##ज',
 '##दा',
 'वय',
 '##वस',
 '##था',
 '##बाट',
 'नया',
 '##य',
 'र',
 'समता',
 'सम',
 '##भव',
 'छ',
 'कसरी',
 'भन',
 '##न',
 'सकिएला']

In [52]:
# text = 'अनि तेस्रो चिन्ता मौसम परिवर्तनले हिमशिखरहरूमा परेका आघातसँगसँगै सिमानावारिपारि नदीले ल्याएका प्रकोपहरू कसरी सम्हाल्ने'
# marked_text = " [CLS] "+text+" [SEP] "
# tokenized_text = tokenizer.tokenize(marked_text)
# indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# segments_ids = [1] * len(indexed_tokens)

# tokens_tensors = torch.tensor([indexed_tokens])
# segments_tensors = torch.tensor([segments_ids])

In [53]:
# with torch.no_grad():
#     outputs = model(tokens_tensors, segments_tensors)
#     hidden_states = outputs.hidden_states
# #     print(hidden_states[-1])
#     token_embeddings = hidden_states[-1]
    
#     token_embeddings = torch.squeeze(token_embeddings, dim = 0)
    
#     list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]
#     print(list_token_embeddings)

In [13]:
texts = ['तर','दुधमा तर बसेन|','तिम्रो घर आउन मन लाग्छ तर अल्छि लाग्छ']

In [14]:
def bert_text_preparation(text, tokenizer ):
    """Preparing input for BERT"""
    
    marked_text = " [CLS] " + text + " [SEP] "
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(indexed_tokens) 
    
    # Convert inputs to Pytorch tensors
    tokens_tensors = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    return tokenized_text, tokens_tensors, segments_tensors

In [15]:
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    # Gradient claculation id disabled 
    # Model is in inference mode
    
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # removing the first hidden state
        # the first state is the input state 
        hidden_states = outputs.hidden_states
    
    # Getting embeddings from final Bert Layer
    tokens_embeddings = hidden_states[-1]
    # Collasping the tensor into 1-dimension 
    tokens_embeddings = torch.squeeze(tokens_embeddings, dim = 0)
    # Converting torchtensors to lists 
    list_token_embeddings = [token_embed.tolist() for token_embed in tokens_embeddings]
    
    return list_token_embeddings 

In [16]:
# tokenized_text, tokens_tensors, segments_tensors = bert_text_preparation(text, tokenizer)
# target_word_embeddings = []

# for text in texts:
#     tokenized_text, tokens_tensors, segments_tensors = bert_text_preparation(text, tokenizer)
#     list_token_embeddings = get_bert_embeddings(tokens_tensors, segments_tensors, model)
#     ## list_token_embeddings has embeddings of the given words
# #     word_index = tokenized_text.index('तर')
#     word_embeddings = [list_token_embeddings[token] for token in tokenized_text]
# #     word_embedding = list_token_embeddings[word_index]
# #     print(word_embedding)
# #     target_word_embeddings.append(word_embedding)

In [17]:
# tokenized_text

In [18]:
# type(tokenized_text[0])

In [19]:
target_word_embeddings = []

for text in texts:
    tokenized_text, tokens_tensors, segments_tensors = bert_text_preparation(text, tokenizers)
    list_token_embeddings = get_bert_embeddings(tokens_tensors, segments_tensors, model)
#     print(len(list_token_embeddings))
    ## list_token_embeddings has embeddings of the given words
    word_index = tokenized_text.index('तर')
    word_embedding = list_token_embeddings[word_index]
#     print(word_embedding)
    target_word_embeddings.append(word_embedding)

In [20]:
target_word_embeddings[0] == target_word_embeddings[1]

False

In [21]:
len(target_word_embeddings)

3

In [22]:
# list_of_distances = []
# for text1, embed1 in zip(texts, target_word_embeddings):
#     for text2, embed2 in zip(texts, target_word_embeddings):
#         cos_dist = 1 - cosine(embed1,embed2)
#         list_of_distances.append([text1, text2, cos_dist])


# distances_df = pd.DataFrame(list_of_distances, columns = ['text1','text2','distance'])


In [7]:
df = pd.read_csv("finalData.csv")

In [8]:
# for txt in df['text'].values:
#     print(len(txt.split('|')))

In [9]:
# def get_word_embeddings(text):
#     tokenizer = tokenizers
#     tokenized_text, tokens_tensors, segments_tensors = bert_text_preparation(text, tokenizer)
#     list_token_embeddings = get_bert_embeddings(tokens_tensors, segments_tensors, model)
#     ## list_token_embeddings has embeddings of the given words
#     return list_token_embeddings

In [10]:
stopwords= stopwords.words("nepali")

In [11]:
words = ['अक्सर','आदि','कसरी','अन्तर्गत','अर्थात','अर्थात्','अलग','आयो','उदाहरण','एकदम','राम्रो','बिरुद्ध','बिशेष']

In [12]:
stopwords = list(set(stopwords).difference(set(words)))

In [13]:
def clean_text(text):
    text = text.split(' ')
    clean_text_list = []
    for word in text:
        if word not in stopwords:
            clean_text_list.append(word)
    clean_text = ' '.join(clean_text_list)
    txt = re.sub(r"[|a-zA-z.'#0-9@,:?'\u200b\u200c\u200d!/&~-]",'',clean_text)
    
    return txt

In [14]:
clean_text("घाम जति लग्यो हामीलाई तेती राम्रो हुन्छ ")

'घाम जति लग्यो हामीलाई तेती राम्रो '

In [15]:
# df['text'] = df['text'].apply(clean_text)

In [16]:
# distances_df[distances_df.text1 == 'तर']

In [17]:
df.head()

,text,label
0,दले ले देश सकेछन सबै बेचे र खान सुरू गरेछन अब ...,0
1,नेपाल को ससकृती ध्वस्त पार्ने योजना हो यो !,0
2,मठ मन्दिर गुम्बा का जग्गा हरु मा भुमाफिया को न...,0
3,नेपाल का कल कर्खाना र नदि नाला बेची सके अब मठ ...,0
4,नेपाल का कल कर्खाना र नदि नाला बेची सके अब मठ ...,0


In [18]:
def get_bert_embedding_sentence(input_sentence):
    md = model
    tokenizer = tokenizers
    marked_text = " [CLS] " + input_sentence + " [SEP] "
    tokenized_text = tokenizer.tokenize(marked_text)

    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(indexed_tokens) 
    
    # Convert inputs to Pytorch tensors
    tokens_tensors = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    with torch.no_grad():
        outputs = md(tokens_tensors, segments_tensors)
        # removing the first hidden state
        # the first state is the input state 

        hidden_states = outputs.hidden_states
#         print(hidden_states[-2])
        # second_hidden_states = outputs[2]
    # hidden_states has shape [13 x 1 x 22 x 768]

    # token_vecs is a tensor with shape [22 x 768]
#     token_vecs = hidden_states[-2][0]
    # get last four layers
#     last_four_layers = [hidden_states[i] for i in (-1,-2, -3,-4)]


    # cast layers to a tuple and concatenate over the last dimension
#     cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
#     print(cat_hidden_states.shape)
    token_vecs = hidden_states[-2][0]

    # take the mean of the concatenated vector over the token dimension
#     sentence_embedding = torch.mean(cat_hidden_states, dim=0).squeeze()

    # Calculate the average of all 22 token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
#     sentence_embedding = torch.mean(token_vecs, dim=1)
    return sentence_embedding.numpy()

In [19]:
get_bert_embedding_sentence("नेपाल को ससकृती ध्वस्त पार्ने योजना")

array([ 2.91070808e-03,  2.76484340e-01,  4.30843160e-02,  2.34462410e-01,
        1.93695560e-01,  8.08570310e-02,  1.67143330e-01, -1.21491715e-01,
        1.98826849e-01, -2.85235912e-01,  8.43921080e-02,  4.07810569e-01,
       -9.35231745e-02,  2.51754194e-01, -2.34684926e-02,  2.99455486e-02,
        1.88694060e-01, -3.43165576e-01, -1.53445810e-01,  3.77470285e-01,
        1.72779873e-01, -1.76885217e-01,  1.78813875e-01, -4.30221707e-01,
        9.03602988e-02, -1.43045172e-01,  3.37987170e-02,  2.06439018e-01,
        6.37543276e-02,  2.21105024e-01,  1.67193666e-01,  4.74679112e-01,
        3.89955603e-02, -5.72416723e-01, -2.64643639e-01, -5.66417277e-01,
        1.11003801e-01,  4.15074170e-01, -2.21883193e-01,  8.65614712e-02,
        1.92130357e-02,  2.18924120e-01,  4.32191640e-01, -9.03057978e-02,
        7.46854618e-02,  3.52693260e-01, -4.17815417e-01,  3.04982275e-01,
       -2.55210251e-01,  1.57838315e-01, -3.86268824e-01, -1.11495972e+00,
        3.70744258e-01, -

In [20]:
df['word_embeddings'] = df['text'].apply(get_bert_embedding_sentence)

In [21]:
df.head()

,text,label,word_embeddings
0,दले ले देश सकेछन सबै बेचे र खान सुरू गरेछन अब ...,0,"[0.12212254, 0.7608289, 0.2592004, -0.22691977..."
1,नेपाल को ससकृती ध्वस्त पार्ने योजना हो यो !,0,"[-0.23879674, 0.24500687, 0.1483417, 0.0913316..."
2,मठ मन्दिर गुम्बा का जग्गा हरु मा भुमाफिया को न...,0,"[-0.029842352, 0.50058687, 0.14775877, 0.36790..."
3,नेपाल का कल कर्खाना र नदि नाला बेची सके अब मठ ...,0,"[-0.042951472, 0.23612754, -0.019368768, 0.340..."
4,नेपाल का कल कर्खाना र नदि नाला बेची सके अब मठ ...,0,"[-0.042951472, 0.23612754, -0.019368768, 0.340..."


In [54]:
# df.to_csv('embedding_data.csv',index = False)

In [10]:
X,y = df['word_embeddings'], df['label']

In [58]:
# scaler = StandardScaler()
# pca = PCA(n_components = 768)

In [59]:
# scaled_X = scaler.fit_transform(X.tolist())
# pca_X = pca.fit_transform(scaled_X)

In [79]:
numpy.array(X[0])

array([ 9.00031477e-02,  1.66499496e-01, -3.07980943e-02, -2.14085490e-01,
       -9.73105878e-02,  7.02001035e-01,  2.27083519e-01, -4.40519065e-01,
       -9.80149746e-01, -1.32598355e-01, -6.32048547e-01,  8.25029686e-02,
        5.32249212e-01, -5.74132621e-01, -8.16702470e-02,  3.62493336e-01,
        1.53544441e-01,  1.80434734e-02,  1.14439704e-01,  2.15428352e-01,
       -3.33787411e-01,  4.09833729e-01, -5.68685532e-01, -3.67090881e-01,
       -5.66257060e-01, -3.14845145e-02, -3.09746414e-01, -1.04817890e-01,
        4.21759933e-01, -1.69556841e-01,  2.54257262e-01, -5.29032826e-01,
        3.64158675e-02,  1.32116973e-01,  1.40526503e-01,  5.25993049e-01,
        4.77683470e-02, -1.94929205e-02, -1.96502596e-01,  2.55206108e-01,
        5.47504425e-02, -1.25304654e-01,  1.82015821e-01, -1.02168359e-01,
       -2.98573077e-01,  4.88719136e-01,  3.07476103e-01,  4.39761765e-02,
       -2.83473074e-01, -6.35397956e-02,  5.71784496e-01, -7.38795474e-02,
       -2.74728835e-01, -

In [13]:
train_X, test_X, train_y, test_y = train_test_split(X,y, test_size = 0.2, random_state = 420)

In [14]:
svc = SVC()

In [82]:
# train_X = [i[0] for i in train_X]
# test_X = [i[0] for i in test_X]

In [83]:
# train_X[0][0].shape

In [15]:
svc.fit(train_X.tolist(), train_y)
#svc.fit(train_X, train_y)

SVC()

In [16]:
svc_pred = svc.predict(test_X.tolist())
# svc_pred = svc.predict(test_X)

In [17]:
print(confusion_matrix(test_y, svc_pred))

[[521 108]
 [190 390]]


In [18]:
print(classification_report(test_y, svc_pred))

              precision    recall  f1-score   support

           0       0.73      0.83      0.78       629
           1       0.78      0.67      0.72       580

    accuracy                           0.75      1209
   macro avg       0.76      0.75      0.75      1209
weighted avg       0.76      0.75      0.75      1209



In [19]:
accuracy_score(test_y, svc_pred)

0.7535153019023987

In [20]:
f1_score(test_y, svc_pred)

0.7235621521335807

In [49]:
nb = GaussianNB()

In [50]:
nb.fit(train_X.tolist(), train_y)
# nb.fit(train_X, train_y)

GaussianNB()

In [51]:
nb_pred = nb.predict(test_X.tolist())
# nb_pred = nb.predict(test_X)

In [52]:
print(confusion_matrix(test_y, nb_pred))

[[518 111]
 [277 303]]


In [53]:
print(classification_report(test_y, nb_pred))

              precision    recall  f1-score   support

           0       0.65      0.82      0.73       629
           1       0.73      0.52      0.61       580

    accuracy                           0.68      1209
   macro avg       0.69      0.67      0.67      1209
weighted avg       0.69      0.68      0.67      1209



In [54]:
accuracy_score(test_y, nb_pred)

0.6790736145574855

In [55]:
# svc.predict(np.array(get_bert_embedding_sentence("नराम्रो").tolist()).reshape(1,-1))

In [56]:
# np.array(get_bert_embedding_sentence("मलाई यो घर राम्रो लागेन ").tolist()).reshape(1,-1).shape

In [57]:
# len(get_bert_embedding_sentence("कति राम्री केटि हो").tolist())

In [58]:
rfc = RandomForestClassifier()

In [59]:
rfc.fit(train_X.tolist(),train_y)
# rfc.fit(train_X,train_y)

RandomForestClassifier()

In [60]:
rfc_predict = rfc.predict(test_X.tolist())
# rfc_predict = rfc.predict(test_X)

In [61]:
print(classification_report(test_y, rfc_predict))

              precision    recall  f1-score   support

           0       0.71      0.72      0.72       629
           1       0.69      0.68      0.68       580

    accuracy                           0.70      1209
   macro avg       0.70      0.70      0.70      1209
weighted avg       0.70      0.70      0.70      1209



In [62]:
gfc = GradientBoostingClassifier()

In [63]:
gfc.fit(train_X.tolist(), train_y)
# gfc.fit(train_X, train_y)

GradientBoostingClassifier()

In [64]:
gfc_predict = gfc.predict(test_X.tolist())
# gfc_predict = gfc.predict(test_X)

In [65]:
print(classification_report(test_y, gfc_predict))

              precision    recall  f1-score   support

           0       0.68      0.72      0.70       629
           1       0.68      0.64      0.66       580

    accuracy                           0.68      1209
   macro avg       0.68      0.68      0.68      1209
weighted avg       0.68      0.68      0.68      1209



In [66]:
gfc.predict(np.array(get_bert_embedding_sentence("त चोर होस्").tolist()).reshape(1,-1))

array([0])

In [67]:
rfc.predict(np.array(get_bert_embedding_sentence("नेपालमा शान्ति छ ").tolist()).reshape(1,-1))

array([1])

In [35]:
# svc.predict(pca.transform(scaler.transform(get_bert_embedding_sentence("एस्तो सोच राख्ने मान्छे त मर्नु बरु ").reshape(1,-1))))
svc.predict(np.array(get_bert_embedding_sentence(" एस्तो सोच  राख्ने मान्छे मरे नि").tolist()).reshape(1,-1))

array([0])

In [69]:
clean_text("एस्तो सोच राख्ने मान्छे त मर्नु बरु")

'एस्तो सोच राख्ने मान्छे मर्नु'

In [70]:
f1_score(test_y,svc_pred)

0.686292548298068

In [71]:
f1_score(test_y,nb_pred)

0.6096579476861167

In [72]:
f1_score(test_y,gfc_predict)

0.6578249336870027

In [73]:
f1_score(test_y,rfc_predict)

0.6841186736474695

In [181]:
10300*12*5

618000

## Keras 

In [53]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasClassifier 
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [64]:
md = Sequential()
md.add(Dense(256, input_dim =X[0].shape[0]))
md.add(Dense(128, activation = 'relu'))
md.add(Dense(12, activation = 'relu'))
md.add(Dense(1, activation = 'sigmoid'))
md.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [65]:
md.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 256)               196864    
                                                                 
 dense_12 (Dense)            (None, 128)               32896     
                                                                 
 dense_13 (Dense)            (None, 12)                1548      
                                                                 
 dense_14 (Dense)            (None, 1)                 13        
                                                                 
Total params: 231,321
Trainable params: 231,321
Non-trainable params: 0
_________________________________________________________________


In [66]:
md.fit(np.asarray([np.asarray(i) for i in train_X]),np.asarray(train_y),epochs = 100, batch_size =70, verbose = 1)

Epoch 1/100
70/70 [==============================] - 0s 2ms/step - loss: 0.5711 - accuracy: 0.7057
Epoch 2/100
70/70 [==============================] - 0s 2ms/step - loss: 0.5095 - accuracy: 0.7479
Epoch 3/100
70/70 [==============================] - 0s 2ms/step - loss: 0.4869 - accuracy: 0.7559
Epoch 4/100
70/70 [==============================] - 0s 2ms/step - loss: 0.4655 - accuracy: 0.7804
Epoch 5/100
70/70 [==============================] - 0s 2ms/step - loss: 0.4512 - accuracy: 0.7795
Epoch 6/100
70/70 [==============================] - 0s 2ms/step - loss: 0.4136 - accuracy: 0.8062
Epoch 7/100
70/70 [==============================] - 0s 2ms/step - loss: 0.3944 - accuracy: 0.8149
Epoch 8/100
70/70 [==============================] - 0s 2ms/step - loss: 0.3699 - accuracy: 0.8321
Epoch 9/100
70/70 [==============================] - 0s 2ms/step - loss: 0.3376 - accuracy: 0.8480
Epoch 10/100
70/70 [==============================] - 0s 2ms/step - loss: 0.3213 - accuracy: 0.8490
Epoch 11/

70/70 [==============================] - 0s 2ms/step - loss: 0.1049 - accuracy: 0.9363
Epoch 84/100
70/70 [==============================] - 0s 2ms/step - loss: 0.0993 - accuracy: 0.9371
Epoch 85/100
70/70 [==============================] - 0s 2ms/step - loss: 0.1009 - accuracy: 0.9371
Epoch 86/100
70/70 [==============================] - 0s 2ms/step - loss: 0.1005 - accuracy: 0.9357
Epoch 87/100
70/70 [==============================] - 0s 2ms/step - loss: 0.1087 - accuracy: 0.9363
Epoch 88/100
70/70 [==============================] - 0s 2ms/step - loss: 0.1183 - accuracy: 0.9307
Epoch 89/100
70/70 [==============================] - 0s 2ms/step - loss: 0.1126 - accuracy: 0.9326
Epoch 90/100
70/70 [==============================] - 0s 2ms/step - loss: 0.1053 - accuracy: 0.9380
Epoch 91/100
70/70 [==============================] - 0s 2ms/step - loss: 0.1042 - accuracy: 0.9338
Epoch 92/100
70/70 [==============================] - 0s 2ms/step - loss: 0.1028 - accuracy: 0.9349
Epoch 93/100


In [416]:
# md.predict(np.asarray(get_bert_embedding_sentence("यो घरको वास्तुशास्त्र मिलेको छैन").tolist()))

In [67]:
md.predict(np.asarray(get_bert_embedding_sentence("मलाई एस्तो बकवास चलचित्र मन्पर्दैन ").tolist()).reshape(1,-1))

array([[0.9705255]], dtype=float32)

In [68]:
result = md.predict(np.asarray([np.asarray(i) for i in test_X]))

In [74]:
res = [0 if i[0]<0.56 else 1 for i in result]

In [75]:
print(f1_score(test_y, res))

0.7063903281519861


In [76]:
print(classification_report(test_y, res))

              precision    recall  f1-score   support

           0       0.73      0.73      0.73       629
           1       0.71      0.71      0.71       580

    accuracy                           0.72      1209
   macro avg       0.72      0.72      0.72      1209
weighted avg       0.72      0.72      0.72      1209



In [1]:
10300*60

618000

In [2]:
118000/500000

0.236